# File Analysis Toolbox

<table table style="float:left; border:none; background-color: #ffffff">
    <tr style="border:none; background-color: #ffffff">
       <td style="float: left">
           <a href="http://pyviz.org/tutorial/07_Geographic_Data.html">     
           <img src="img/l_bokeh.png" style="width:50px; align: left;"></a>    
       </td>
       <td style="border:none; float: left">
           <h2>Geo Plotting</h2>
       </td>
    </tr>
    <tr style="border:none; background-color: #ffffff">
       <td style="border:none; float: left">
           <a href="https://pandas.pydata.org">     
           <img src="img/l_pandas.png" style="height:50px; align: left"></a>
       </td>
       <td style="border:none; float: left">
           <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html">
           <img src="img/l_excel.png" style="height:50px; align: center"></a>
       </td>
       <td style="border:none; float: left">
           <a href="https://www.mysql.com/products/connector/">
           <img src="img/l_mysql.png" style="height:50px; align: center"></a>    
       </td>
       <td style="border:none; float: left">
           <h2>Dataframes</h2>
       </td>
    </tr>
    <tr style="border:none; background-color: #ffffff">
       <td style="border:none; float: left">
           <a href="https://3.python-requests.org">
           <img src="img/l_req.png" style="width:50px; align: left;"></a>
       </td>
       <td style="border:none; float: left">
           <h2>HTTP Get/Put</h2>
       </td>
    </tr>    
    <tr style="border:none; background-color: #ffffff">
       <td style="border:none; float: left">
           <a href="https://github.com/cghall/force-retrieve">     
           <img src="img/l_sfdev.png" style="height:50px; align: left;"></a>    
       </td>
       <td style="border:none; float: left">
           <a href="http://developer.pardot.com/kb/api-version-3/opportunities/">     
           <img src="img/l_pardot.png" style="height:50px; align: left;"></a>
       </td>
       <td style="border:none; float: left">
           <h2>REST APIs</h2>
       </td>
    </tr>
</table>

In [21]:
import io
import sys
import numpy as np
import pandas as pd
from collections import OrderedDict
import xlrd                                                #!{sys.executable} -m pip install xlrd
import IPython
import bokeh
from bokeh.io import output_notebook, output_file, show, push_notebook
from bokeh.plotting import *
from bokeh.models import ColumnDataSource, HoverTool, CategoricalColorMapper
from bokeh.layouts import row, column, gridplot, widgetbox
from bokeh.models.widgets import Tabs, Panel
from bokeh.transform import factor_cmap
import hvplot as hv
import hvplot.pandas
from simple_salesforce import Salesforce                   # !{sys.executable} -m pip install salesforce-reporting
from pardot.client import APIClient                        # !{sys.executable} -m pip install pardot-api-client
import pytz
import datetime
from uszipcode import SearchEngine                         # !{sys.executable} -m pip install uszipcode
search = SearchEngine(simple_zipcode=False)                # set simple_zipcode=False to use rich info database
!pip3 install --upgrade uszipcode                          # upgrade databases
from progressbar import ProgressBar
print("python:", sys.version)
print("numpy:", np.__version__)
print("pandas:", pd.__version__)
print("xlrd:", xlrd.__version__)
print("IPython:", IPython.__version__)
print("bokeh:", bokeh.__version__)
print("holoviews:", hv.__version__)
print("Salesforce:", Salesforce)
print("Pardot:", APIClient)
output_notebook()
from simple_salesforce import Salesforce                   # !{sys.executable} -m pip install simple-salesforce
from salesforce_reporting import Connection, ReportParser  # !{sys.executable} -m pip install salesforce_reporting
with open('.sfdc') as f:
    uname, spass, stoken, ppass, ptoken = [x.strip("\n") for x in f.readlines()]

Requirement already up-to-date: uszipcode in /usr/local/lib/python3.8/site-packages (0.2.4)
python: 3.8.5 (default, Jul 21 2020, 10:48:26) 
[Clang 11.0.3 (clang-1103.0.32.62)]
numpy: 1.19.1
pandas: 1.1.0
xlrd: 1.2.0
IPython: 7.17.0
bokeh: 2.1.1
holoviews: 0.6.0
Salesforce: <class 'simple_salesforce.api.Salesforce'>
Pardot: <class 'pardot.client.APIClient'>


Loading BokehJS ...

In [4]:
search = SearchEngine(simple_zipcode=True)                # False uses rich info database
zipcode = search.by_zipcode("06916")
# zipcode.to_json()                                       # to json
zipcode.to_dict()                                         # to dict
# zip = zipcode.values()                                  # to list
zipcode

NameError: name 'SearchEngine' is not defined

<a href="https://github.com/cghall/force-retrieve"><img src="img/l_sf.png" style="width:80px; float: left;"></a> 
<h2 style="padding-left:100px; border:none">Query</h2>

In [171]:
sf = Salesforce(username=uname, password=spass, security_token=stoken)          # authenticate
end = datetime.datetime.now(pytz.UTC)                                           # salesforce API requires UTC
# print("Session ID:", session_id)
# print("Instance:", sf_instance)
# print("Version:", sf_version)
print("@ ", end)

@  2019-08-19 21:49:20.291462+00:00


In [181]:
acctdf = sf.bulk.Account.query("SELECT Name, Id, OwnerID, BDRAssigned__c FROM Account WHERE CAPDB__c <> '' AND Closed_Won_Opportunity_Count__c = 0 AND of_Active_Opps__c =  0 AND of_Opportunities__c > 0 AND BDRAssigned__c <> 'None Assigned' LIMIT 10")
records = [dict(
        Id = rec['Id'],
   OwnerID = rec['BDRAssigned__c'])
    for rec in query['records']]
acctdf = pd.DataFrame(records)
acctdf.to_csv (r'CAPDB_CL.csv', index = None, header=True)

# acctdf_j = acctdf.to_json(orient='records')
# acctdf_j
# sf.bulk.Account.update(acctdf_j)           # sf.Contact.update('003e0000003GuNXAA0',{'LastName': 'Jones', 'FirstName': 'John'})

In [125]:
query = sf.query("SELECT Id, PostalCode, StateCode FROM Lead WHERE LeadSource = 'Madison'")
records = [dict(
            Id = rec['Id'],
    PostalCode = rec['PostalCode'],
     StateCode = rec['StateCode'])
    for rec in query['records']]
qdf = pd.DataFrame(records)
qdf['PostalCode'] = qdf['PostalCode'].str[:5]       # trim off zip+4
def zco(x):
    return search.by_zipcode(x).state
qdf['StateCode'] = qdf['PostalCode'].fillna(0).astype(int).apply(zco)
qdf.to_csv (r'madison.csv', index = None, header=True
# qdfj = qdf.to_json(orient='records')
# qdfj
# qdf
# sf.bulk.Contact.insert(data)
# sf.bulk.Contact.upsert(data, 'Id')
# sf.bulk.Lead.update(qdfj)

<a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html"><img src="img/l_pandas.png" style="height:50px; float: left;"></a> 
<h2 style="padding-left:80px; border:none">CSV IO</h2>

In [25]:
# df = pd.read_csv('data/hosp.csv', index_col='Target?', dtype={'user_id': int})
# df = pd.read_csv('data/15Pct.csv')
# with open('csvfile.csv', 'w') as csvfile: f = csv.writer(csvfile) f.writerows(items)  # write to csv file

<a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html"><img src="img/l_excel.png" style="width:50px; float: left;"></a> 
<h2 style="padding-left:80px; border:none">Excel Methods</h2>

In [26]:
# df = pd.read_excel('data/15Pct.xlsx', '15Pct')
# df.dtypes
# writer = pd.ExcelWriter('examples/ex2.xlsx')
# df.to_excel(writer, 'Sheet1')
# writer.save()
# df.to_excel('examples/ex2.xlsx')
# !rm examples/ex2.xlsx

<a href="https://pandas.pydata.org"><img src="img/l_pandas.png" style="width:70px; float: left;"></a>    
<h2 style="padding-left:100px; border:none">DataFrames</h2>

--------------------------------------  
  
### attributes  
  
```python
DataFrame.shape                         # dimensions (tuple)
DataFrame.describe()                    # shows a quick statistic data summary
DataFrame.info()                        # data types and specs
DataFrame.dtypes                        # column labels & data types
DataFrame.index                         # index (row labels)
DataFrame.head(n)                       # return first n rows
DataFrame.tail(n)                       # return last n rows    
DataFrame.columns                       # column labels
DataFrame.values                        # Numpy representation
DataFrame.axes                          # list axes
DataFrame.size                          # number (int) elements
DataFrame.memory_usage([index, deep])   # each column memory (bytes)
DataFrame.empty                         # empty
DataFrame.is_copy                       # return copy
```
  
### plots
Callable method and namespace attribute for specific plotting methods
```python
DataFrame.plot([x, y, kind, ax, ….])	    # DataFrame plotting accessor and method
DataFrame.plot.area([x, y])	                # Draw a stacked area plot.
DataFrame.plot.bar([x, y])	                # Vertical bar plot.
DataFrame.plot.barh([x, y])	                # Make a horizontal bar plot.
DataFrame.plot.box([by])	                # Make a box plot of the DataFrame columns.
DataFrame.plot.density([bw_method, ind])	# Generate Kernel Density Estimate plot using Gaussian kernels.
DataFrame.plot.hexbin(x, y[, C, …])	        # Generate a hexagonal binning plot.
DataFrame.plot.hist([by, bins])	            # Draw one histogram of the DataFrame’s columns.
DataFrame.plot.kde([bw_method, ind])	    # Generate Kernel Density Estimate plot using Gaussian kernels.
DataFrame.plot.line([x, y])	                # Plot DataFrame columns as lines.
DataFrame.plot.pie([y])	                    # Generate a pie plot.
DataFrame.plot.scatter(x, y[, s, c])	    # Create a scatter plot with varying marker point size and color.
DataFrame.boxplot([column, by, ax, …])	    # Make a box plot from DataFrame columns.
DataFrame.hist([column, by, grid, …])	    # Make a histogram of the DataFrame’s.
```

  
### transforms
```python
DataFrame.isna()	                            # Detect missing values.
DataFrame.between_time(start_time, end_time)	# Select between times of the day (e.g., 9:00-9:30 AM)
DataFrame.drop([labels, axis, index, …])	    # Drop specified labels from rows or columns
DataFrame.set_index(keys[, drop, append, …])	# Set the DataFrame index using existing columns
  
DataFrame.['col0'].astype(str)                  # change dtype (int32, int64, str, bool, category
numpy.integer,signedinteger.int8,int16,int32,int64,timedelta64,
numpy.unsignedinteger.uint8,.uint16,.uint32,.uint64,.uint64
numpy.floating.float16,float32,float64,float128
numpy.complexfloating.complex64, complex128, complex256
numpy.flexible.character
numpy.bytes_, numpy.str_
numpy.void
numpy.record
numpy.bool_,
numpy.datetime64,
numpy.object_
  
DataFrame = DataFrame.set_index("col0")
DataFrame.drop("Ireland", axis=0)                            # Delete all rows with label "Ireland"
```

   
### sort
```python
DataFrame.sort_index(axis=2, ascending=False)                 # axis sort
DataFrame.sort_values(by='Target?')                           # value sort 'column'
```
map() method produces complex criteria
```python
criterion = DataFrame['a'].map(lambda x: x.startswith('t'))   # only want 'two' or 'three'
DataFrame[criterion]
```
```python
DataFrame[[x.startswith('t') for x in DataFrame['a']]]        # equivalent but slower
DataFrame[criterion & (DataFrame['b'] == 'x')]                # with multiple criteria
DataFrame.loc[criterion & (DataFrame['b'] == 'x'), 'b':'c']   # Select by Label, Position, Advanced Indexing
```


### lookup()
Extract a set of values given a sequence of row labels and column labels return **NumPy array**

```python
DataFrame.lookup(list(range(0, 10, 2)), ['B', 'C', 'A', 'B', 'D'])
```


### query()

column b has values between column a and c values
```python
DataFrame.query('(a < b) & (b < c)')
index = pd.MultiIndex.from_arrays([colors, foods], names=['color', 'food'])
```

columns a and "b" have overlapping values
```python
DataFrame.query('a in b')
```

columns a and b have overlapping values and col c's values are less than col d's
```python
DataFrame.query('a in b and c < d')
```

Comparing a list of values to a column using ==/!= works similarly to in/not in.
```python
DataFrame.query('b == ["a", "b", "c"]')
```


### duplicate()
remove duplicate rows

*duplicated returns boolean vector len = #rows indicating duplicated rows*
```python
DataFrame.duplicated('a', keep=False)
DataFrame.duplicated(['a', 'b'])
```
+ drop_dupes
```python
DataFrame.drop_duplicates('a', keep='last')
```

dupe handling
+ keep='first' (default): mark / drop duplicates except for the first occurrence
+ keep='last': mark / drop duplicates except for the last occurrence
+ keep=False: mark / drop all duplicates

drop dupes
```python
DataFrame.index.duplicated()
DataFrame[~DataFrame.index.duplicated()]
```
```python
DataFrame[~DataFrame.index.duplicated(keep='last')]
DataFrame[~DataFrame.index.duplicated(keep=False)]
```


### df.loc()  
    
```python
DataFrame.loc[<row selection>, <column selection>]
```  
 
set index on _col0_ column  
```python
DataFrame.set_index('col0', inplace=True)
```  
  
select rows with index values 'Andrade' + 'Veness' with columns fr 'city' to 'email'  
```python
nDataFrame.loc[['Andrade', 'Veness'], 'city':'email']
```  
  
select same rows, with just 'first_name', 'address' and 'city' columns  
```python
DataFrame.loc['Andrade':'Veness', ['first_name', 'address', 'city']]
```  
  
select rows with _first name_ Antonio and _columns_ 'city' to 'email'  
  
```python
DataFrame.loc[DataFrame['first_name'] == 'Antonio', 'city':'email']
```  
  
Select rows where email column ends w/ 'hotmail.com' include all columns  
```python
DataFrame.loc[DataFrame['email'].str.endswith("hotmail.com")]
```  
  
  
### indexing isin()  
  
returns boolean vector true where series elements exist in passed list  
row select with selected row/column values  
  
```python
s = pd.Series(np.arange(5), index=np.arange(5)[::-1], dtype='int64')
df = pd.DataFrame({'vals': [1, 2, 3, 4], 'ids': ['a', 'b', 'f', 'n'],'ids2': ['a', 'n', 'c', 'n']})
values = ['a', 'b', 1, 3]
values = {'ids': ['a', 'b'], 'vals': [1, 3]}                  # match values with columns into dict
df.isin(values)
```

*combine DataFrame isin() any() all() methods to select data subsets meeting criteria. Select rows where column meets its own criterion:*
```python
values = {'ids': ['a', 'b'], 'ids2': ['a', 'c'], 'vals': [1, 3]}
row_mask = df.isin(values).all(1)
df[row_mask]
df.query('a < b < c')
```

### <a href="https://developer.salesforce.com/docs/api-explorer"><img src="img/l_sfdev.png" style="width:200px; float: left;"></a>    
<h2 style="padding-left:225px; border:none">Salesforce Methods</h2>
  
--------------------------------------  

Create file /root/.sfdc with credentials as:  
username  
password  
token  
uid
instance

### authenticate 
```python
from salesforce_reporting import Connection, ReportParser
with open('.sfdc') as f:
    username, password, token, uid, instance = [x.strip("\n") for x in f.readlines()]
sf = Salesforce(username=username, password=password, security_token=token, sandbox=False)
```

### query
```python
ContactDict = sf.query("SELECT Company, CreatedDate, Email, FirstName, Industry, LastName, LeadSource, Title FROM Lead ORDER BY CreatedDate DESC NULLS FIRST LIMIT 50")

opportunity_amounts = sf.query("SELECT Id, Probability, StageName, Amount, Owner.Name FROM Opportunity WHERE AMOUNT < 10000")

amounts = [x['Amount'] for x in opportunity_amounts['records']]
owners = [x['Owner']['Name'] for x in opportunity_amounts['records']]
hist1 = go.Histogram(x=amounts)
py.iplot([hist1], filename='salesforce/opportunity-probability-histogram')
```

sf.Contact.updated(end - datetime.timedelta(days=10), end)       # contacts updated last 10d
contact = sf.Contact.get('003e0000003GuNXAA0')                   # 18 digit id
metadata = sf.Contact.metadata()
describe = sf.Contact.describe()

### read into dataframe
```python
df = pd.DataFrame(ContactDict['records'])
df.head(10)
Age = df['Age__c']
Age.head(10)
agePlot = Age.hist(bins=10)
```

### process
```python
lead_comparison = []
for name, vals in df.groupby('Owners'):
    counts = vals.Status.value_counts()
    lead_comparison.append(Bar(x=counts.index, y=counts.values, name=name))

scatters = []
for name, temp_df in large_opps_df.groupby('Owner'):
    hover_text = temp_df.Name + "<br>Close Probability: " + temp_df.Probability.map(str) + "<br>Stage:" + temp_df.StageName
    scatters.append(
        go.Scatter(
            x=temp_df.CloseDate,
            y=temp_df.Amount,
            mode='markers',
            name=name,
            text=hover_text,
            marker=dict(
                size=(temp_df.Probability / 2) # helps keep the bubbles of managable size
            )
        )
    )

data = scatters
layout = go.Layout(
    title='Open Large Deals',
    xaxis=dict(
        title='Close Date'
    ),
    yaxis=dict(
        title='Deal Amount',
        showgrid=False
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='salesforce/open-large-deals-scatter')
```

### reports

```python
!pip install salesforce-analytics
```

[salesforce-reporting](https://github.com/cghall/force-retrieve)

```python
!pip install salesforce-reporting

report = sf.get_report('report_id', includeDetails=True)
parser = salesforce_reporting.ReportParser(report)
parser.records()
```
  
```python
report = my_sf.get_report('report_id')
matrix_parser = salesforce_reporting.MatrixParser(report)
matrix_parser.series_down('Jan 2016')
```

### Salesforce Streaming Client
A Salesforce streaming API client for python, built on salesforce-requests-oauthlib and python-bayeux.

To run tests, install py.test and pytest-cov in your virtualenv

py.test --cov=src/salesforce_streaming_client/ --cov-report html:coverage  

View test coverage results ./coverage

```python  
pip3 install salesforce-streaming-client
```  

### [Pardot](https://github.com/mneedham91/PyPardot4)

Version 4 for duped email Pardot configurations

**Install**  
_Requires [requests](http://docs.python-requests.org/en/latest/)_

```python
$ !pip install pypardot4
```
  
  
|Parameter|	Required|Description|  
|---------|---------|-----------|  
|object|X|The object type to be returned by the API request|  
|op|X|The operation to be performed on the specified object type|  
|id_field|X|The field to be used as the identifier for the specified object|  
|id|X|The identifier for the specified object(s)|  
|your_api_key|X|The API key that was obtained during Authentication|  
|your_user_key|X|The user key that was used during Authentication|  
|format|The API data format. Either xml or json (xml is default)|  
|params|Parameters specific to your request; See individual methods for details|  

**Authenticate**
```python
pa = PardotAPI(email='email',password='password',user_key='user_key')  
pa.authenticate()
```

**Create a new prospect**
```python
pa.prospects.create(email='joe@company.com', first_name='Joe', last_name='Schmoe')
```

**Read data about our prospect**
```python
print(pa.prospects.read_by_email(email='joe@company.com'))
```

**Query and iterate through today's prospects**
Pardot only returns 200 records with each request  
Use offset to retrieve matching records beyond this limit  

```python
prospects = pa.prospects.query(created_after='yesterday')
total = prospects['total_results'] # total number of matching records
for prospect in prospects['prospect']
  print(prospect.get('first_name'))
```
  
**Create a new prospect**
```python
pa.prospects.create_by_email(email='joe@company.com', first_name='Joe', last_name='Schmoe')
```

**Update a prospect field (works with default or custom field)**
```python
pa.prospects.update_field_by_id(id=23839663, field='company', field_value='Joes Plumbing')
```

**Send a one-off email**
```python
pa.emails.send_to_email(prospect_email='joe@company.com', email_template_id=123)
```

**Performing API calls is inherently unsafe, so be sure to catch exceptions**
```python
try:
  pa.prospects.create_by_email(email='existing.email.address@company.com')
except PardotAPIError, e:
  print(e)
```

### [Pardot](https://github.com/joshgeller/PyPardot)

_version 3 for non duping email configuration_

```python
$ !pip install pypardot
from pypardot.client import PardotAPI
with open(.pardot) as f:
    username, password, token = [x.strip("\n") for x in f.readlines()]
pa = PardotAPI(email=username, password=password, user_key=token)
pa.authenticate()
prospects = pa.prospects.query(created_after='yesterday')
prospects
```  
  
  
### [Pardot](https://github.com/maxnaude/pardot-api-client)

```python
!pip3 install pardot-api-client

from pardot.client import APIClient
with open('.pardot') as f:
    username, password, token = [x.strip("\n") for x in f.readlines()]
client = APIClient(username, password, token)
pa = client.prospect.read('someone@example.com')
pa = client.prospect.update('someone@example.com', first_name='John')
for r in client.prospect.query(new=True): print(r)
```

--------------------------------------  

In [85]:
from pardot.client import APIClient

with open('.pardot') as f:
    username, password, token = [x.strip("\n") for x in f.readlines()]
client = APIClient(username, password, token)
pa = client.prospect.read('someone@example.com')
# pa = client.prospect.update('someone@example.com', first_name='John')
# for r in client.prospect.query(new=True): print(r)

<a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html"><img src="img/l_excel.png" style="width:70px; float: left;"></a>    
<h1 style="padding-left:125px; border:none">Pandas Excel Methods</h1> 

--------------------------------------  
```python  
$ !pip install xlrd  
import xlrd
xlsx = pd.ExcelFile('examples/ex1.xlsx')  
pd.read_excel(xlsx, 'Sheet1')  
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')  
frame  
writer = pd.ExcelWriter('examples/ex2.xlsx')  
frame.to_excel(writer, 'Sheet1')  
writer.save()  
frame.to_excel('examples/ex2.xlsx')  
!rm examples/ex2.xlsx  
```
--------------------------------------  

<a href="https://www.mysql.com/products/connector/"><img src="img/l_mysql.png" style="width:100px; float: left;"></a>    
<h1 style="padding-left:125px; border:none">MySQL Methods</h1>
  
--------------------------------------  
```python
!pip install mysql-connector-python
import mysql.connector
config = {
    'host': 'rpsmithii.mysql.pythonanywhere-services.com',
    'database': 'rpsmithii$weight','user': 'rpsmithii',
    'password': 'Home@Call4','port': '3306'}
db = mysql.connector.connect(**config)
cur = db.cursor()
cur.execute("SELECT dt, wht FROM weight WHERE wht > 10")
table = pd.DataFrame(cur.fetchall())
table.columns = cur.column_names
```
--------------------------------------  


In [ ]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [2]:
a = 2
b = 6
a + b

8